# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097016


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:41,  1.48s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:03,  2.35s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.70s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:30,  1.25s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:12<00:01,  4.28it/s]

Rendering models:  87%|████████▋ | 26/30 [00:12<00:00,  4.48it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  4.80it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  4.62it/s]

not-logged-in-025360e89444727733a2    0.000486
Jnursssmith5263                       0.001760
jnarayanbvg                           0.007653
not-logged-in-c6917d9b19f41e6c2995    0.000755
awright5                              0.001873
not-logged-in-0c0cc122e942f046bb82    0.000964
rmz4160                               0.000474
ethanscamander                        0.002450
Dynamatt                              0.000312
asucich                               0.000286
not-logged-in-f7e2260ed4a9517a3e4a    0.003133
not-logged-in-f7e2260ed4a9517a3e4a    0.017355
not-logged-in-ebe11e1994328fe49a89    0.000424
ElisabethB                            0.001688
not-logged-in-d0bc4156ef5274edc132    0.167388
not-logged-in-8aba8657ce3870731948    0.001997
sn346808                              0.001010
not-logged-in-693ab1c69c45f852d665    0.005196
LeeJoseph                             0.015967
not-logged-in-154b91cc2bc572b9f687    0.001833
mami2019                              0.001318
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())